# Inferring the rotation period of [KIC 1430163](https://archive.stsci.edu/kepler/data_search/search.php?target=1430163&action=Search&outputformat=HTML_Table) <br>from [Kepler](https://archive.stsci.edu/kepler/) using [<i>george</i>](https://github.com/dfm/george) for a custom kernel

Based on the [revised version](https://github.com/Paveloom/C2/blob/master/Notebooks/Base/celerite/Revised/revised.ipynb) of the [example](https://github.com/dfm/celerite/blob/master/paper/figures/rotation/rotation.ipynb) from Foreman-Mackey ([2017](https://www.mendeley.com/catalogue/fast-scalable-gaussian-process-modeling-applications-astronomical-time-series/)). Written by [Pavel Sobolev](https://github.com/Paveloom) and located [here](https://github.com/Paveloom/C2). Release version: [0.2.1](https://github.com/Paveloom/C2/releases/tag/v0.2.1).

Importing standard modules:

In [ ]:
import re  # Regular expression operations
import copy  # Deep copy operations

Importing non-standard packages and modules:

[kplr](https://github.com/dfm/kplr), [corner](https://github.com/dfm/corner.py), [emcee](https://github.com/dfm/emcee), and [george](https://github.com/dfm/george) developed by [Daniel Foreman-Mackey](https://github.com/dfm) and other contributors;<br>
[matplotlib](https://github.com/matplotlib/matplotlib) developed by [Michael Droettboom](https://github.com/mdboom),  [Thomas Caswell](https://github.com/tacaswell) and other contributors;<br>
[autograd](https://github.com/HIPS/autograd) developed by [Dougal Maclaurin](https://dougalmaclaurin.com), [David Duvenaud](https://www.cs.toronto.edu/~duvenaud/), [Matt Johnson](http://people.csail.mit.edu/mattjj/), [Jamie Townsend](https://github.com/j-towns) and other contributors;<br>
[optimization](https://github.com/scipy/scipy/tree/master/scipy/optimize) module from [scipy](https://github.com/scipy/scipy) developed by contributors;<br>
[timeseries](https://github.com/astropy/astropy/tree/master/astropy/timeseries) module from [astropy](https://github.com/astropy/astropy) developed by contributors.

In [ ]:
import kplr  # Tools for working with Kepler data
import corner  # Tools for making corner plots
import emcee  # The Python ensemble sampling toolkit for affine-invariant MCMC

from emcee import interruptible_pool as ip  # Interruptible Multiprocessing Pool
from emcee import autocorr as ac  # Autocorrelation Function

# Fast and flexible Gaussian Process regression in Python
import george  # Full interface
from george import kernels  # Kernels interfaces

# Plots
from matplotlib import rcParams as rcP  # Plot parameters
from matplotlib import pyplot as plt  # 2D Plots
from celerite import plot_setup  # Plot setup

from autograd import numpy as np  # Derivatives of numpy code
from scipy.optimize import minimize  # Minimization of a scalar function
from astropy.timeseries import LombScargle  # Computing of the Lomb-Scargle Periodogram

Setting plot parameters:

In [ ]:
# A magic command to get inline plots within a Python Notebook
%matplotlib inline

# A magic command to enable 2x plots
%config IPython.matplotlib.backend = "retina"

# Setting the DPIs of figures
rcP["savefig.dpi"] = 300
rcP["figure.dpi"] = 300

# Setting the Computer Modern Bright font
rcP["text.usetex"] = True
rcP["text.latex.preamble"] = r"\usepackage{cmbright}"

# Setting the font sizes
rcP["font.size"] = 16
rcP["legend.fontsize"] = 12

Loading sample data from the [Kepler Input Catalog](https://archive.stsci.edu/kepler/kic.html) (field descriptors are taken from [Kepler Archive Manual](https://archive.stsci.edu/files/live/sites/mast/files/home/missions-and-data/kepler/_documents/archive_manual.pdf)):

In [ ]:
# The ID of a star from the Kepler Input Catalog
KIC_ID = 1430163

# Attaching the kplr API
client = kplr.API()

# Getting the star data
star = client.star(KIC_ID)

# Time values
t = []

# Pre-search Data Conditioning Simple Aperture Photometry (PDCSAP) flux values
f = []

# The 1-sigma errors in PDC flux values
ferr = []

# Getting long cadence data of light curves
for lc in star.get_light_curves(short_cadence=False):

    # Reading data from a light curve
    data = lc.read()

    # Getting time values from the data
    t0 = data["TIME"]

    # Getting PDCSAP flux values from the data
    f0 = data["PDCSAP_FLUX"]

    # Creating a mask for only qualitative data points
    mask = (data["SAP_QUALITY"] == 0) & np.isfinite(t0) & np.isfinite(f0)

    # Appending the qualitative time data to the existing list
    t.append(t0[mask])

    # Computing the median of the flux data
    mu = np.median(f0[mask])

    # Scaling the qualitative flux data and appending these to the existing list
    f.append((f0[mask] / mu - 1.0) * 1e3)

    # Scaling the qualitative flux errors data and appending these to the existing list
    ferr.append(data["PDCSAP_FLUX_ERR"][mask] / mu * 1e3)

# Concatenating the arrays within the lists

t = np.concatenate(t)
f = np.concatenate(f)
ferr = np.concatenate(ferr)

# Getting contiguous arrays

t = np.ascontiguousarray(t, dtype=float)
f = np.ascontiguousarray(f, dtype=float)
ferr = np.ascontiguousarray(ferr, dtype=float)

Making a first guess at the period using the Lomb-Scargle periodogram (see, e.g VanderPlas ([2017](https://www.mendeley.com/catalogue/understanding-lombscargle-periodogram-1/))) <br>and comparing it to the real value known from Mathur et al. ([2014](https://www.mendeley.com/catalogue/investigating-magnetic-activity-f-stars-kepler-mission/)):

In [ ]:
# Creating a set of evenly spaced frequencies
freq = np.linspace(1e-1, 1.0, 5000)

# Getting the PSD-normalized Lomb-Scargle power spectrum of the data using the fast method (O[N log N])
power = LombScargle(t, f).power(freq, method="fast", normalization="psd")

# Determining the maximum peak
period = 1.0 / freq[np.argmax(power)]

# Printing the x value of this peak
print("The maximum peak is at " + str(period))

# Scaling the output
power /= len(t)

# Plotting the result with emphasis on the period value
# and comparing it to the real value

# Plotting data
plt.plot(1.0 / freq, power, color="#425378")

# Marking the true period value
plt.axvline(3.88, color="#7C40A0", alpha=0.5)

# Adding approximate period values on the plot
plt.text(3.9, 0.0006, r"$\approx$ 3.88 days", fontsize=12)
plt.text(4.385, 0.0006, r"$\approx$ 4.358 days", fontsize=12)

# Setting limits
plt.xlim(period - 1, period + 1)
plt.ylim(0, 0.0007)

# Setting labels
plt.xlabel("Period, days")
plt.ylabel("Power")

# Setting title
plt.title("Lomb-Scargle periodogram")

# Saving the figure
plt.savefig("Figures/LS_periodogram", bbox_inches="tight")

# Showing the result
plt.show()

# Closing the plot
plt.close()

Taking a subset of the data (here: quarters [4](https://archive.stsci.edu/kepler/preview.php?type=lc&dsn=KPLR001430163-2010078095331) and [5](https://archive.stsci.edu/kepler/preview.php?type=lc&dsn=KPLR001430163-2010174085026) of the long cadence data):

In [ ]:
# Defining a mask with time bounds
mask = (t >= 352.396596505) & (t <= 537.5502295)

# Taking the subset
t = t[mask]
f = f[mask]
ferr = ferr[mask]

Defining the covariance function as described in the example from Foreman-Mackey ([2017](https://www.mendeley.com/catalogue/fast-scalable-gaussian-process-modeling-applications-astronomical-time-series/)):

replicating the properties of

$ \LARGE k(\tau) = A \exp\left( - \frac{\tau^2}{2 l^2} - \Gamma \sin^2\left( \frac{\pi \tau}{P_{rot}} \right) \right)$

by constructing a simple <i>celerite</i> covariance function

$ \LARGE k(\tau) = \frac{a}{2 \ + \ b} e^{-\tau/c} \left[ \cos\left( \frac{2 \pi \tau}{P} \right) + (1 + b) \right] $

The parameters are defined by logarithms in the code for the simplicity of specifying priors.

[Custom kernel](https://github.com/Paveloom/C2/tree/master/Notebooks/Base/celerite/Kernel/RotationKernel.yml) is compiled locally since george [doesn't support](https://george.readthedocs.io/en/latest/tutorials/new-kernel/#new-kernel) in-place defining because of technical limitations.

In [ ]:
# Defining the priors
bounds = dict(log_amp=(-10.0, 0.0), log_timescale=(1.5, 5.0),
              log_period=(-3.0, 5.0), log_factor=(-5.0, 5.0))

# Building a kernel
kernel = kernels.RotationKernel(
    log_amp=np.log(np.var(f)),
    log_timescale=np.log(10.0),
    log_period=np.log(period),
    log_factor=np.log(1.0),
    bounds=bounds
)

Creating a Gaussian process model with the specified kernel; defining the functions 
for computing negative marginalized log likelihood and the gradient of it:

In [ ]:
# Building a celerite model with the specified kernel
gp = george.GP(kernel, mean=np.median(f))

# Computing the extended form of the covariance matrix and factorizing
gp.compute(t, ferr)


def neg_log_like(params, y, model, m):
    """
    
    A function to get the negative marginalized log likelihood

    Parameters:
        params: An array containing parameter values;
        y : An array of data;
        model : A GP model;
        m : A mask for an array of data

    Returns:
        The negative marginalized log likelihood
        
    """

    # Setting parameter values to the given vector
    model.set_parameter_vector(params)

    # Computing and returning the result
    return -model.log_likelihood(y[m])


def grad_neg_log_like(params, y, model, m):
    """
    
    A function to get the gradient of
    the negative marginalized likelihood 

    Parameters:
        params: An array containing parameter values;
        y : An array of data;
        model : A GP model;
        m : A mask for an array of data

    Returns:
        The gradient of the negative
        marginalized likelihood
        
    """

    # Setting parameter values to the given vector
    model.set_parameter_vector(params)

    # Computing and returning the result
    return -model.grad_log_likelihood(y[m])

Step-by-step $\Large\sigma$-clipping:

In [ ]:
# Getting the values of the initial parameters
init_params = gp.get_parameter_vector()

# Getting the names of the parameters
params_names = [re.sub(r".*:", "", name) for name in gp.get_parameter_names()]

# Getting bounds values
bounds_values = gp.get_parameter_bounds()

# Creating an initial mask
sigma_mask = np.ones(len(t), dtype=bool)

gp.compute(t[sigma_mask], ferr[sigma_mask])
print(grad_neg_log_like(init_params, f, gp, sigma_mask))

# Main loop
while False:

    # Computing the extended form of the covariance matrix and factorizing
    gp.compute(t[sigma_mask], ferr[sigma_mask])

    # Minimizing the negative marginalized likelihood using the L-BFGS-B method
    opt = minimize(neg_log_like, init_params, jac=grad_neg_log_like,
                   method="L-BFGS-B", bounds=bounds_values, args=(f, gp, sigma_mask))

    # Setting the new parameters
    gp.set_parameter_vector(opt.x)

    # Computing the conditional predictive distribution of the model
    mu, var = gp.predict(f[sigma_mask], t, return_var=True)

    # Calculating standard deviation
    sig = np.sqrt(var + ferr ** 2)

    # Calculating the residuals
    rds = f - mu

    # Calculating a new mask
    new_sigma_mask = np.abs(rds) < 3 * sig

    # Comparing the masks
    if np.all(new_sigma_mask == sigma_mask):
        break

    # Updating the mask
    sigma_mask[:] = new_sigma_mask

# Saving the arrays using the computed mask
fit_t, fit_f, fit_ferr = t[sigma_mask], f[sigma_mask], ferr[sigma_mask]

# Copying ``gp``
ml_gp = copy.deepcopy(gp)

# Computing the extended form of the covariance matrix and factorizing
ml_gp.compute(fit_t, fit_ferr)

# Computing the marginalized log likelihood
ml_gp.log_likelihood(fit_f)

# Printing the optimal parameters

print("The maximum a posteriori parameters (MAP) inferred by\n",
      "minimizing the negative marginalized likelihood: \n", sep="")

for i in range(len(params_names)):
    print(params_names[i] + ":", opt.x[i])

In [ ]:
[-2395.6919298   2386.93237776   632.25350827    71.56259315]